In [1]:
import matplotlib.pyplot as plt
from datasets.PupilCoreDatasetPupil import PupilCoreDataset
import torch
from eye_model import EyeModeling
from torchvision.models.segmentation import FCN_ResNet50_Weights, fcn_resnet50
import numpy as np
import cv2
import utils
from models.ifOpened import ifOpenedModel
from models.trainers import PupilSegmentationTrainer, IfOpenedTrainer

In [2]:
DATASET_LEN_TO_USE = 5000
dataset = PupilCoreDataset(
    "datasets/PupilCoreDataset/video5_eye0_video.avi",
    'datasets/PupilCoreDataset/video5_eye0_pupildata.csv',
    "datasets/PupilCoreDataset/video5_eye1_video.avi",
    'datasets/PupilCoreDataset/video5_eye1_pupildata.csv',
    DATASET_LEN_TO_USE
)
# dataset.get_pupil_ellipse()
# dataset.save_masks("datasets/PupilCoreDataset/created_masks")

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
ifOpenedModel = ifOpenedModel()
PupilSegmentationModel = fcn_resnet50(weights=None, num_classes=1)

c:\Users\Admin\Desktop\inzynierka\my_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\Admin\Desktop\inzynierka\my_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
dataset.load_masks("datasets/PupilCoreDataset/created_masks/eye0", "datasets/PupilCoreDataset/created_masks/eye1")

In [6]:
torch.set_printoptions(edgeitems=100)


In [7]:
if_opened_trainer = IfOpenedTrainer(model=ifOpenedModel, dataset=dataset, dataset_len=DATASET_LEN_TO_USE)
pupil_trainer = PupilSegmentationTrainer(model=PupilSegmentationModel, dataset=dataset, dataset_len=DATASET_LEN_TO_USE)

In [8]:
# dataset_pupil.get_pupil_ellipse()

In [9]:
# pupil_trainer.train(device=device, num_epochs=5)

In [10]:
# pupil_trainer.train(device=device, num_epochs=5)
pupil_trainer.model.load_state_dict(torch.load("models/weights/resnet50.pt"))


<All keys matched successfully>

In [11]:
def visualize_mask(image, mask):
    with torch.no_grad():
        pupil_trainer.model.eval()
        pupil_trainer.model = pupil_trainer.model.to(device)
        for i, (inputs, masks, opened) in enumerate(pupil_trainer.dataloaders['test']):

            inputs = inputs.to(device)
            outputs = pupil_trainer.model(inputs)


            image = np.transpose(inputs[0].cpu().numpy(), (1, 2, 0)).copy()
            outputs_sig = torch.sigmoid(outputs['out'][0])
            outputs_sig = np.transpose(outputs_sig.cpu().numpy(), (1, 2, 0)).copy()

            # visualize_pupil(image, outputs_sig)
            ellipse = utils.fit_ellipse(outputs_sig)
            print(ellipse)
            break



In [12]:
# TODO zmierzyc focal_len na obrazkach w pikselach
# trzeba zebrac obrazki gdzie oko patrzy w miare prosto
masks_idx_to_get_pupil_radius = [1186, 1614, 1795, 3096, 3137, 3252, 3320, 4246]
# plt.imshow(dataset.eye0_frames[1186])
# plt.show()
# plt.imshow(dataset.eye0_masks[1186])
# plt.show()
# plt.imshow(dataset.eye0_masks[1614])
# plt.show()
# plt.imshow(dataset.eye0_masks[1795])
# plt.show()
# plt.imshow(dataset.eye0_masks[3096])
# plt.show()
# plt.imshow(dataset.eye0_masks[3137])
# plt.show()
masks_for_radius = [dataset.eye0_masks[idx] for idx in masks_idx_to_get_pupil_radius]
utils.get_pupil_radius_from_images(masks_for_radius)
# utils.fit_ellipse(masks_to_get_pupil_radius[0])


((115.97880554199219, 100.99464416503906), (21.899005889892578, 26.178293228149414), 97.20674896240234)
((98.01856231689453, 60.05283737182617), (26.295475006103516, 27.625457763671875), 120.30876922607422)
((67.0, 54.0), (20.054094314575195, 22.13465690612793), 5.476513862609863)
((105.000732421875, 101.08638000488281), (20.21428108215332, 23.93794822692871), 111.64768981933594)
((105.02687072753906, 100.02754211425781), (20.20345115661621, 26.08974266052246), 105.77178192138672)
((125.01477813720703, 96.9703140258789), (21.947195053100586, 25.969667434692383), 87.8128890991211)
((123.98171997070312, 98.00631713867188), (24.02872085571289, 27.988697052001953), 91.60010528564453)
((112.05626678466797, 81.9906234741211), (26.168251037597656, 27.963890075683594), 106.60002136230469)


In [13]:
# TODO swirski test
focal_len = 140 # ??? taken from https://github.com/pupil-labs/pupil/blob/master/pupil_src/shared_modules/camera_models.py Pupil Cam3
eye_modeling = EyeModeling(ellipse=ellipse, focal_len=focal_len)

NameError: name 'ellipse' is not defined

In [ ]:
#TODO 2 NN system
